In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import Demand
import Solar
import Price
import Date

In [2]:
def analysis(bigdata):

    eff = 0.9
    batterycharge = 0
    chargeneeded = 0
    median = np.median(bigdata['Grid Buy Price'])
    

    for n in range(0,24):
        
        if bigdata['Net Demand'][n] <= 0:
            batterycharge = batterycharge - bigdata['Net Demand'][n]*eff
            
        if bigdata['Grid Buy Price'][n] >= median*1.5:
            chargeneeded = chargeneeded + bigdata['Net Demand'][n]/eff
            

    return batterycharge, chargeneeded


In [3]:
## method to run the battery ##

def run_battery(bigdata):
    
    batteryoutput = np.zeros(24)
    soc = np.ones(24)*0
    capacity = 750
    power = 420
    eff = 0.9


    for n in range(0,24):
        if n == 0:
            soc_temp = 0
        else:
            soc_temp = soc[n-1]

        if bigdata['Net Demand'][n] > 0:   #using the battery
            batteryoutput[n] = min(bigdata['Net Demand'][n], soc_temp*eff, power)
            soc[n] = soc_temp - batteryoutput[n]*(1/eff)

        elif bigdata['Net Demand'][n] < 0:     # charge the battery, NOTE NET DEMAND = -VE
            batteryoutput[n] = max(bigdata['Net Demand'][n], -(1/eff)*(capacity - soc_temp), -power)
            soc[n] = soc_temp - batteryoutput[n]*(eff)

        elif bigdata['Net Demand'][n] == 0:      # do nothing
            soc[n] = soc_temp


    ## adds the battery output to total demand and adds to bigdata ##

    bigdata['Net Demand - battery'] = bigdata['Net Demand']-batteryoutput
    bigdata['Running Cost - battery'] = bigdata['Net Demand - battery']*bigdata['Grid Buy Price']
    bigdata['Running Cost - battery'] = bigdata['Running Cost - battery'].clip(lower=0)
    bigdata['Running Profit - battery'] = -bigdata['Net Demand - battery']*bigdata['Grid Sell Price']
    bigdata['Running Profit - battery'] = bigdata['Running Profit - battery'].clip(lower=0)
    bigdata['soc - battery'] = soc
    bigdata['battery output - battery'] = batteryoutput
    
    #bigdata = bigdata.round(3)
    ## returns the new bigdata ##
    
    return bigdata

In [4]:
## method to run the SMART system ##

def run_SMART(bigdata):
    
    batteryoutput = np.zeros(24)
    soc = np.ones(24)*0
    capacity = 750
    power = 420
    eff = 0.9
    
    median = np.median(bigdata['Grid Buy Price'])
    predicted_battery_charge, required_battery_charge = analysis(bigdata)

    for n in range(0,24):
        
        if n == 0:
            soc_temp = 0
        else:
            soc_temp = soc[n-1]

        ## the smart design ##
        ## the smart design ##
        ## the smart design ##

        
        if required_battery_charge > predicted_battery_charge: 
            if n == 5:
                batteryoutput[n] = max(-power, -(1/eff)*(required_battery_charge - predicted_battery_charge))
                soc[n] = soc_temp - batteryoutput[n]*(1/eff)
                continue
                
        if n == 0:
            use = False
        if bigdata['Grid Buy Price'][n] >= median*1.5:
            use = True
            

        ## the smart design ##
        ## the smart design ##
        ## the smart design ##

        if bigdata['Net Demand'][n] > 0:   #using the battery
            batteryoutput[n] = min(bigdata['Net Demand'][n], soc_temp*eff, power)
            soc[n] = soc_temp - batteryoutput[n]*(1/eff)
            if use == False:
                soc[n] = soc[n-1]
                batteryoutput[n] = 0

        elif bigdata['Net Demand'][n] < 0:     # charge the battery
            batteryoutput[n] = max(bigdata['Net Demand'][n], -(1/eff)*(capacity - soc_temp), -power)
            soc[n] = soc_temp - batteryoutput[n]*(eff)

        elif bigdata['Net Demand'][n] == 0:      # do nothing
            soc[n] = soc_temp

        
    ## adds the battery output to total demand and adds to bigdata ##

    bigdata['Net Demand - SMART'] = bigdata['Net Demand']-batteryoutput
    bigdata['Running Cost - SMART'] = bigdata['Net Demand - SMART']*bigdata['Grid Buy Price']
    bigdata['Running Cost - SMART'] = bigdata['Running Cost - SMART'].clip(lower=0)
    bigdata['Running Profit - SMART'] = -bigdata['Net Demand - SMART']*bigdata['Grid Sell Price']
    bigdata['Running Profit - SMART'] = bigdata['Running Profit - SMART'].clip(lower=0)
    bigdata['soc - SMART'] = soc
    bigdata['battery output - SMART'] = batteryoutput
    
    #bigdata = bigdata.round(3)
    ## returns the new bigdata ##
    
    return bigdata


In [5]:
## plots the graphs ##

def plotgraphs(bigdata):
    
    x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

    #############.  Net Demand: No battery vs battery    ##########
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,7))
    ax1.step(x, bigdata['Net Demand'], color = 'red', label = 'raw demand')
    ax1.grid()
    ax1.set_xlabel('Time throughout the day')
    ax1.set_ylabel('(kWh)')
    ax1.set_xticks((0,5,11,17,23))
    ax1.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    ax2.step(x, bigdata['Net Demand - battery'], color = 'blue', label = 'demand with battery')
    ax2.grid()
    ax2.set_xlabel('Time throughout the day')
    ax2.set_ylabel('(kWh)')
    ax2.set_xticks((0,5,11,17,23))
    ax2.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    fig.suptitle('Net Demand: No battery vs battery', fontsize=20)
    fig.legend(fontsize=12)


    #############.  Net Demand: No battery vs SMART    ##########
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,7))
    ax1.step(x, bigdata['Net Demand'], color = 'red', label = 'raw demand')
    ax1.grid()
    ax1.set_xlabel('Time throughout the day')
    ax1.set_ylabel('(kWh)')
    ax1.set_xticks((0,5,11,17,23))
    ax1.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    ax2.step(x, bigdata['Net Demand - SMART'], color = 'blue', label = 'demand with SMART')
    ax2.grid()
    ax2.set_xlabel('Time throughout the day')
    ax2.set_ylabel('(kWh)')
    ax2.set_xticks((0,5,11,17,23))
    ax2.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    fig.suptitle('Net Demand: No battery vs SMART', fontsize=20)
    fig.legend(fontsize=12)


    #############.  Net Demand: battery vs SMART    ##########
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,7))
    ax1.step(x, bigdata['Net Demand - battery'], color = 'red', label = 'demand with battery')
    ax1.grid()
    ax1.set_xlabel('Time throughout the day')
    ax1.set_ylabel('(kWh)')
    ax1.set_xticks((0,5,11,17,23))
    ax1.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    ax2.step(x, bigdata['Net Demand - SMART'], color = 'blue', label = 'demand with SMART')
    ax2.grid()
    ax2.set_xlabel('Time throughout the day')
    ax2.set_ylabel('(kWh)')
    ax2.set_xticks((0,5,11,17,23))
    ax2.set_xticklabels(('00:00','06:00','12:00','18:00','23:00'))
    fig.suptitle('Net Demand: No battery vs SMART', fontsize=20)
    fig.legend(fontsize=12)

In [6]:
def summary(bigdata):
    
            print('no battery running cost is £' +str(bigdata['Running Cost'].sum()))
            print('with battery running cost is £' +str(bigdata['Running Cost - battery'].sum()))
            print('with SMART running cost is £' +str(bigdata['Running Cost - SMART'].sum()))
            print('no battery Profit is £' +str(bigdata['Running Profit'].sum()))
            print('with battery Profit is £' +str(bigdata['Running Profit - battery'].sum()))
            print('with SMART Profit is £' +str(bigdata['Running Profit - SMART'].sum()))
 

In [7]:
def year_summary(no_battery_cost,yes_battery_cost,SMART_battery_cost,no_battery_profit,yes_battery_profit,SMART_battery_profit):
    
    print('no battery running cost is £' +str(no_battery_cost.sum()))
    print('with battery running cost is £' +str(yes_battery_cost.sum()))
    print('with SMART running cost is £' +str(SMART_battery_cost.sum()))
    print('no battery Profit is £' +str(no_battery_profit.sum()))
    print('with battery Profit is £' +str(yes_battery_profit.sum()))
    print('with SMART Profit is £' +str(SMART_battery_profit.sum()))


In [8]:
def costs(bigdata):
    
    return bigdata['Running Cost'].sum(), bigdata['Running Cost - battery'].sum(), bigdata['Running Cost - SMART'].sum()

In [9]:
def profit(bigdata):
    
    return bigdata['Running Profit'].sum(), bigdata['Running Profit - battery'].sum(), bigdata['Running Profit - SMART'].sum()

# input the date you want here

In [10]:
datewanted = "01/04/2019"    ###  month day year  ###

# input the date you want here

In [11]:
## initialises the daily cost vectors ##
no_battery_cost = np.zeros(365)
yes_battery_cost = np.zeros(365)
SMART_battery_cost = np.zeros(365)
no_battery_profit = np.zeros(365)
yes_battery_profit = np.zeros(365)
SMART_battery_profit = np.zeros(365)


for n in range(0, 365):

    ## loads in the data ##

    daywanted = n+1
    datewanted = Date.getdatewanted(daywanted)
    totaldemand = Demand.gettotaldemand(daywanted)
    solarsupply = Solar.getsolarsupply(daywanted)
    buyprice = Price.getvariablebuyprice(daywanted)
    sellprice = Price.getvariablesellprice(daywanted)
    bigdata =pd.concat([totaldemand, solarsupply, buyprice, sellprice], axis=1)
    bigdata['Net Demand'] = bigdata['Total Demand (kWh)']-bigdata['Total Supply (kWh)']
    bigdata['Running Cost'] = bigdata['Grid Buy Price']*bigdata['Net Demand']
    bigdata['Running Cost'] = bigdata['Running Cost'].clip(lower=0)
    bigdata['Running Profit'] = -bigdata['Grid Sell Price']*bigdata['Net Demand']
    bigdata['Running Profit'] = bigdata['Running Profit'].clip(lower=0)

    ## runs the battery for that day ##

    bigdata = run_battery(bigdata)

    ## runs the SNART system for that day ##

    bigdata = run_SMART(bigdata)

    ## analyses that day ##

    nothing_cost, battery_cost, smart_cost = costs(bigdata)
    nothing_profit, battery_profit, smart_profit = profit(bigdata)

    ## adds the day into the year vector ##

    no_battery_cost[n] = nothing_cost
    yes_battery_cost[n] = battery_cost
    SMART_battery_cost[n] = smart_cost
    no_battery_profit[n] = nothing_profit
    yes_battery_profit[n] = battery_profit
    SMART_battery_profit[n] = smart_profit

In [12]:
year_summary(no_battery_cost,yes_battery_cost,SMART_battery_cost,no_battery_profit,yes_battery_profit,SMART_battery_profit)

no battery running cost is £38169.4776973125
with battery running cost is £17917.85601740925
with SMART running cost is £16162.480315279665
no battery Profit is £102623.56742500002
with battery Profit is £84761.50756083336
with SMART Profit is £84761.50756083336


In [13]:
# yearvar = np.zeros((366,23))
# yearcons = np.zeros((366,23))


# for n in range(0,366):
#     daywanted = n
#     var = Price.getvariablesellprice(daywanted)
#     cons = Price.getconstantsellprice(daywanted)
    
#     for j in range(0,23):
#         yearvar[n,j] = var.iloc[j][0]
#         yearcons[n,j] = cons.iloc[j][0]

# fig, ax = plt.subplots()

# ax.plot(np.average(yearvar, axis=0))
# ax.plot(np.average(yearcons, axis=0))
# ax.set_ylim([0,0.15])